# Generating Subsets of Wikidata

>Warning: 
**This notebook is under construction and it doesn't work**

## Purpose

>This notebook is used to create smaller subgraphs from a larger input Wikidata graph. Notebook users can provide a list of Wikidata classes (**QNodes**) to remove and preserve to create desired subsets of Wikidata. 


### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

UPDATE EXAMPLE INVOCATION


```
papermill Wikidata\ Useful\ Files.ipynb useful-files.out.ipynb \
-p wiki_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.wikibase-item.tsv.gz \
-p property_item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.property.wikibase-item.tsv.gz \
-p qual_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/qual.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no \
-p compute_pagerank no \
-p languages es,ru,zh-cn 
```

In [1]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import papermill as pm

import gzip

In [76]:
# Parameters

# Folder on local machine where to create the output and temporary folders
# output_path = "/Users/pedroszekely/Downloads/kypher"
# output_path = "/Users/markmann/Downloads/subset"
output_path = "/nas/home/mbmann/subset"

# The names of the output and temporary folders
output_folder = "output"
temp_folder = "temp.output"

# Classes to remove
#Q34508 - video tape recording
# remove_classes = "Q13442814, Q523, Q16521, Q318, Q7318358, Q7187, Q11173, Q8054, Q5, Q13100073, Q8502, Q3305213, Q4022, Q79007, Q1931185, Q30612, Q101352, Q54050, Q13433827, Q2668072, Q23397, Q3863, Q11424, Q482994, Q47150325, Q16970, Q18593264, Q355304, Q9842, Q7725634, Q27020041, Q56436498, Q2154519, Q61443690, Q49008, Q3331189, Q47521, Q5084, Q19389637, Q21014462, Q4164871, Q11060274, Q5633421, Q39816, Q5185279, Q55488, Q134556, Q22698, Q985488, Q1260524, Q204107, Q2225692, Q215380, Q71963409, Q452237, Q93184, Q12323"
# remove_classes = "Q34508"

# The location of input files
# wiki_root_folder = "/Volumes/GoogleDrive/Shared\ drives/KGTK/datasets/wikidata-20200803-v4/"
# wiki_root_folder = "/Volumes/GoogleDrive/Shared\ drives/KGTK/datasets/wikidata-20200803-v4/"
# wiki_root_folder = "/Users/pedroszekely/Downloads/kypher/wikidataos-v4/"
# wiki_root_folder = "/Users/markmann/Google\ Drive/Shared\ drives/KGTK/datasets/wikidataos-v4-mm-2/"
wiki_root_folder = "/nas/home/mbmann/kgtk/datasets/wikidataos-v4-mm-2/"

metadata_folder = "/nas/home/mbmann/kgtk/datasets/wikidata-20200803-v5/data/"

claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"
qual_file = "qualifiers.tsv.gz"
property_datatypes_file = "metadata.property.datatypes.tsv.gz" #FIX
metadata_file = "metadata.types.tsv.gz" #FIX
isa_file = "derived.isa.tsv.gz"
p279star_file = "derived.P279star.tsv.gz"

# Useful files Jupyter notebook
useful_files_notebook = "Wikidata Useful Files.ipynb"
# notebooks_folder = "/Users/markmann/Desktop/CKG/kgtk_subset/kgtk/examples/"
notebooks_folder = "/nas/home/mbmann/kgtk_subset/kgtk/examples/"

# Location of the cache database for kypher
# cache_path = "/Users/pedroszekely/Downloads/kypher/wikidataos-v4"
cache_path = f'{output_path}/{output_folder}'

#Additional parameters
delete_database = "no"
compute_pagerank = "no"
languages = ""

### Needs fixing
# Whether to delete the cache database
if delete_database and delete_database.lower().strip() == 'yes':
    delete_database = True
else:
    delete_database = False

### Needs fixing
if compute_pagerank and compute_pagerank.lower().strip() == 'yes':
    compute_pagerank = True
else:
    compute_pagerank = False

if languages:
    languages = languages.split(',')

## Set up variables for files

In [77]:
#Environment variables
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

#Python variables
if cache_path:
    store = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    store = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

out = "{}/{}".format(output_path, output_folder)
temp = "{}/{}".format(output_path, temp_folder)

claims = wiki_root_folder + claims_file
labels = wiki_root_folder + label_file
aliases = wiki_root_folder + alias_file
descriptions = wiki_root_folder + description_file
items = wiki_root_folder + item_file
quals = wiki_root_folder + qual_file
isa = wiki_root_folder + isa_file
p279star = wiki_root_folder + p279star_file

datatypes = metadata_folder + property_datatypes_file #FIX
metadata = metadata_folder + metadata_file #FIX

# shortcuts to commands
kgtk_path = "~/anaconda3/envs/kgtk-subset/bin/kgtk"
kgtk = f'time {kgtk_path} --debug'
kypher = f"{kgtk_path} query --debug --graph-cache " + store

Go to the output directory and create the subfolders for the output files and the temporary files

In [4]:
!cd $output_path
!mkdir {out}
!mkdir {temp}

mkdir: cannot create directory ‘/nas/home/mbmann/subset/output’: File exists
mkdir: cannot create directory ‘/nas/home/mbmann/subset/temp.output’: File exists


Clean up the output and temp folders before we start

In [5]:
# !rm {out}/*.tsv {out}/*.tsv.gz
# !rm {temp}/*.tsv {temp}/*.tsv.gz

if delete_database:
    !rm {out}/*.tsv {out}/*.tsv.gz
    !rm {temp}/*.tsv {temp}/*.tsv.gz

### Preview the input files

It is always a good practice to peek a the files to make sure the column headings are what we expect

In [6]:
!{kypher} -i {claims} \
--match '()-[]->()' \
--limit 10

[2021-02-11 13:58:00 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"	normal	url
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	normal	url
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	normal	wikibase-item
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	normal	wikibase-property
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	normal	wikibase-property
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	normal	wikibase-property
P10-P1659-P51-86aca4c5-0	P10	P1659	P51	normal	wikibase-property
P10-P1855-Q7378-555592a4-0	P10	P1855	Q7378	normal	wikibase-item
P10-P31-Q18610173-85ef4d24-0	P10	P31	Q18610173	normal	wikibase-item
P1000-P1629-Q1241356-d5c10f50-0	P1000	P1629	Q1241356	normal	wikibase-item


## Creating a list of all the items  to remove

**Add classes to remove, based on <u>classes themselves</u> here:** <br>
- **Example:** Let's remove the class (videotape recording, 'Q34508')
- **NOTE:** This will only remove items that have a P31/P279 relation with the class

In [7]:
classes_to_remove = ['Q34508'] #Parameter: Add classes manually here
classes = ', '.join([f'"{c}"' for c in classes_to_remove])

!{kypher} -i {claims} -o {temp}/classes.remove.manual.tsv.gz \
--match '(instance)-[:P279]->(c)' \
--where 'c in [{classes}]' \
--return 'c, instance, "p279" as label'

!zcat {temp}/classes.remove.manual.tsv.gz |head

[2021-02-11 13:58:02 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."node2", graph_1_c1."node1", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     AND (graph_1_c1."node2" IN (?))
  PARAS: ['p279', 'P279', 'Q34508']
---------------------------------------------
node2	node1	label
Q34508	Q1926658	p279
Q34508	Q20892182	p279
Q34508	Q23058567	p279
Q34508	Q25324511	p279
Q34508	Q2916762	p279
Q34508	Q48774253	p279
Q34508	Q97065731	p279
Q34508	Q97073931	p279
Q34508	Q97078183	p279


**Add classes to remove, based on <u>instances</u> here:**
- **Example:** Let's remove classes that are part of instance (Fireball, 'Q5451712'), (Bush, 'Q1017471'), and (Italin Grape Alle, 'Q67772833')
- **NOTE:** The expected class to remove is (whisky, 'Q281'), (beer, 'Q44'), (beer brand, Q15075508), and (ale, 'Q208385')

Specify the intances to remove

In [8]:
instances_to_remove = ['Q5451712', 'Q1017471', 'Q67772833'] #Parameter: Add instances manually here
instances = ', '.join([f'"{instance}"' for instance in instances_to_remove])

For all instances, get their **(P279, subclass)** from `claims.tsv.gz`

In [9]:
!{kypher} -i {claims} -o {temp}/classes.remove.p279.tsv.gz \
--match '(instance)-[:P279]->(c)' \
--where 'instance in [{instances}]' \

!zcat {temp}/classes.remove.p279.tsv.gz |head

[2021-02-11 13:58:34 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     AND (graph_1_c1."node1" IN (?, ?, ?))
  PARAS: ['P279', 'Q5451712', 'Q1017471', 'Q67772833']
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
Q67772833-P279-Q208385-3af88d61-0	Q67772833	P279	Q208385	normal	wikibase-item


For all instances, get their **(P31, class)** from `claims.tsv.gz`

In [10]:
!{kypher} -i {claims} -o {temp}/classes.remove.p31.tsv.gz \
--match '(instance)-[:P31]->(c)' \
--where 'instance in [{instances}]' \

!zcat {temp}/classes.remove.p31.tsv.gz |head

[2021-02-11 13:58:40 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     AND (graph_1_c1."node1" IN (?, ?, ?))
  PARAS: ['P31', 'Q5451712', 'Q1017471', 'Q67772833']
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
Q1017471-P31-Q15075508-61e783df-0	Q1017471	P31	Q15075508	normal	wikibase-item
Q1017471-P31-Q44-7580116c-0	Q1017471	P31	Q44	normal	wikibase-item
Q5451712-P31-Q281-2d4512be-0	Q5451712	P31	Q281	normal	wikibase-item
Q67772833-P31-Q44-2d9b5c1b-0	Q67772833	P31	Q44	normal	wikibase-item


Concatenate all `classes.remove.manual`, `classes.remove.p31`, and `classes.remove.p279` into one file

In [11]:
!{kgtk} cat -i {temp}/classes.remove.manual.tsv.gz -i {temp}/classes.remove.p31.tsv.gz -i {temp}/classes.remove.p279.tsv.gz \
-o {temp}/classes.remove.tsv.gz


real	0m0.665s
user	0m0.304s
sys	0m0.050s


Check and remove duplicate classes. 

In [12]:
!{kgtk} unique -i {temp}/classes.remove.tsv.gz \
-o {temp}/classes.remove2.tsv.gz

!zcat {temp}/classes.remove2.tsv.gz | head


real	0m0.833s
user	0m0.389s
sys	0m0.083s
node1	label	node2
Q15075508	count	1
Q208385	count	1
Q281	count	1
Q34508	count	9
Q44	count	2


In [ ]:
#ISSUE: Can't query for multiple relations for given node1 within same query
#GITHUB: https://github.com/usc-isi-i2/kgtk/issues/330
#Test 1
# !{kypher} -i {claims} \
# --match '(n1)-[:P31]->(n2)' \
# --where 'n1 = "Q5451712"' \

#Test 2, Test 3 
# !{kypher} -i {claims} \
# --match 'claims: (n1)-[l1 {label: p}]->(n2)' \
# --where 'n1 = "Q5451712" and p = "P31"' \
# --where 'n1 = "Q5451712" and p = "P31" OR p = "P279"' \

### Compute the items to be removed via classes

First look at the classes we will remove

In [13]:
!zcat {temp}/classes.remove2.tsv.gz | head

node1	label	node2
Q15075508	count	1
Q208385	count	1
Q281	count	1
Q34508	count	9
Q44	count	2


1. Given all classes in `classes.remove2`, find all subclasses from `p279star`. <br>
2. Given all subclasses from `p279star`, find all subclass instances from `isa`
3. The resulting items to remove will be in `{temp}/items.remove.byclass.tsv.gz`

In [14]:
!{kypher} -i {temp}/classes.remove2.tsv.gz -i {p279star} -i {isa} \
--match 'isa: (item)-[:isa]->(subclass), P279star: (subclass)-[:P279star]->(c), class: (c)-[:count]->()' \
--return 'distinct item, "p31_p279star" as label, c as node2' \
-o {temp}/items.remove.byclass.tsv.gz

[2021-02-11 13:58:50 sqlstore]: DROP graph data table graph_7 from /nas/home/mbmann/subset/temp.output/classes.remove2.tsv.gz
[2021-02-11 13:58:50 sqlstore]: IMPORT graph directly into table graph_11 from /nas/home/mbmann/subset/temp.output/classes.remove2.tsv.gz ...
[2021-02-11 13:58:50 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_4_c1."node1", ? "_aLias.label", graph_3_c2."node2" "_aLias.node2"
     FROM graph_11 AS graph_11_c3, graph_3 AS graph_3_c2, graph_4 AS graph_4_c1
     WHERE graph_11_c3."label"=?
     AND graph_3_c2."label"=?
     AND graph_4_c1."label"=?
     AND graph_11_c3."node1"=graph_3_c2."node2"
     AND graph_3_c2."node1"=graph_4_c1."node2"
  PARAS: ['p31_p279star', 'count', 'P279star', 'isa']
---------------------------------------------
[2021-02-11 13:58:50 sqlstore]: CREATE INDEX on table graph_11 column label ...
[2021-02-11 13:58:50 sqlstore]: ANALYZE INDEX on table graph_11 column label ...
[2021-02-11 13:58:51 

Check the result

In [15]:
# !zcat {temp}/items.remove.byclass.tsv.gz | head
!echo 'Johnnie Walker'
!zgrep 'Q502268	' {temp}/items.remove.byclass.tsv.gz
!echo 'Fireball'
!zgrep 'Q5451712	' {temp}/items.remove.byclass.tsv.gz

Johnnie Walker
Q502268	p31_p279star	Q281
Fireball
Q5451712	p31_p279star	Q281


### Compute the items to be removed via out degree

Specify the # of node out-degrees `k`, and identify items with out-degree less than `k`
- Ex: Find items that have out-degree `k` less than 2.

Compute out-degree for all QNodes in the `claims` file. Check the result.

In [16]:
!{kypher} -i {claims} -o {temp}/metadata.out_degree.tsv.gz \
--match '(n1)-[l]->()' \
--where "upper(substr(n1,0)) >= 'Q'" \
--return 'distinct n1 as node1, count(distinct l) as node2, "out_degree" as label' 

!zcat {temp}/metadata.out_degree.tsv.gz | head

[2021-02-11 13:59:30 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", count(DISTINCT graph_1_c1."id") "_aLias.node2", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     WHERE (upper(substr(graph_1_c1."node1", ?)) >= ?)
     GROUP BY "_aLias.node1"
  PARAS: ['out_degree', 0, 'Q']
---------------------------------------------
node1	node2	label
Q1	81	out_degree
Q1000	559	out_degree
Q100000003	11	out_degree
Q100000011	9	out_degree
Q100000014	9	out_degree
Q100000021	9	out_degree
Q100000029	10	out_degree
Q100000030	2	out_degree
Q100000033	10	out_degree

gzip: stdout: Broken pipe


Create a list of items that have out_degree < `k`, along with any parent classses they belong to. <br>
Put the results into `items.remove.bydegree.tsv.gz`. <br>

In [17]:
k = 2 #Parameter
!{kypher} -i {temp}/metadata.out_degree.tsv.gz -i {isa} -i {p279star} \
--match 'out: (item)-[:out_degree]->(n2), isa: (item)-[:isa]->(subclass), P279star: (subclass)-[:P279star]->(c)' \
--where 'cast(n2, integer) <= {k}' \
--return 'distinct item, "p31_p279star" as label, c as node2' \
-o {temp}/items.remove.bydegree.tsv.gz \

!zcat {temp}/items.remove.bydegree.tsv.gz | head

[2021-02-11 14:00:14 sqlstore]: DROP graph data table graph_8 from /nas/home/mbmann/subset/temp.output/metadata.out_degree.tsv.gz
[2021-02-11 14:00:19 sqlstore]: IMPORT graph directly into table graph_12 from /nas/home/mbmann/subset/temp.output/metadata.out_degree.tsv.gz ...
[2021-02-11 14:00:23 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_4_c2."node1", ? "_aLias.label", graph_3_c3."node2" "_aLias.node2"
     FROM graph_12 AS graph_12_c1, graph_3 AS graph_3_c3, graph_4 AS graph_4_c2
     WHERE graph_12_c1."label"=?
     AND graph_3_c3."label"=?
     AND graph_4_c2."label"=?
     AND graph_12_c1."node1"=graph_4_c2."node1"
     AND graph_3_c3."node1"=graph_4_c2."node2"
     AND (CAST(graph_12_c1."node2" AS integer) <= ?)
  PARAS: ['p31_p279star', 'out_degree', 'P279star', 'isa', 2]
---------------------------------------------
[2021-02-11 14:00:23 sqlstore]: CREATE INDEX on table graph_12 column node1 ...
[2021-02-11 14:00:40 sqlstore]: A

### Combine the items to remove by-class and by-outdegree
Concatenate all items from `items.remove.byclass` and `items.remove.bydegree`.
The resulting list of items to remove will be `items.remove`.

In [18]:
# !zcat {temp}/items.remove.byclass.tsv.gz | head
# !echo 'Johnnie Walker'
# !zgrep 'Q502268	' {temp}/items.remove.byclass.tsv.gz
# !echo 'Fireball'
# !zgrep 'Q5451712	' {temp}/items.remove.byclass.tsv.gz

# !zcat {temp}/items.remove.bydegree.tsv.gz | head
# !zcat {temp}/items.remove.tsv.gz | head

!{kgtk} cat -i {temp}/items.remove.byclass.tsv.gz {temp}/items.remove.bydegree.tsv.gz \
-o {temp}/items.remove.tsv.gz

#Check if fireball is still in there
# !echo 'Fireball'
# !zgrep 'Q5451712	' {temp}/items.remove.tsv.gz


real	2m17.588s
user	2m16.648s
sys	0m0.142s


Deduplicate the concatenated file of items to remove. <br>
The resulting list of items to remove will be `items.remove2`.

In [19]:
!{kypher} -i {temp}/items.remove.tsv.gz -o {temp}/items.remove2.tsv.gz \
--match '(item)-[:p31_p279star]->(c)' \
--return 'distinct item, "p31_p279star" as label, c as node2'
!zcat {temp}/items.remove2.tsv.gz | head

[2021-02-11 14:08:45 sqlstore]: DROP graph data table graph_9 from /nas/home/mbmann/subset/temp.output/items.remove.tsv.gz
[2021-02-11 14:08:54 sqlstore]: IMPORT graph directly into table graph_9 from /nas/home/mbmann/subset/temp.output/items.remove.tsv.gz ...
[2021-02-11 14:09:07 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_9_c1."node1", ? "_aLias.label", graph_9_c1."node2" "_aLias.node2"
     FROM graph_9 AS graph_9_c1
     WHERE graph_9_c1."label"=?
  PARAS: ['p31_p279star', 'p31_p279star']
---------------------------------------------
[2021-02-11 14:09:07 sqlstore]: CREATE INDEX on table graph_9 column label ...
[2021-02-11 14:10:30 sqlstore]: ANALYZE INDEX on table graph_9 column label ...
node1	label	node2
Q1000737	p31_p279star	Q15075508
Q1017471	p31_p279star	Q15075508
Q10350781	p31_p279star	Q15075508
Q10355535	p31_p279star	Q15075508
Q10655045	p31_p279star	Q15075508
Q10774749	p31_p279star	Q15075508
Q1092501	p31_p279star	Q15075508


### Validate the items we will remove
Check the `items.remove` file for classes added via different methods: 1) by-class, 2) by-instance, 3) by-outdegree

1) Check for class added manually, i.e. (videotape recording, 'Q34508')

In [20]:
!echo 'videotape recording'
!zgrep 'Q34508' {temp}/items.remove2.tsv.gz | head

videotape recording
Q11931979	p31_p279star	Q34508
Q100431477	p31_p279star	Q34508
Q100982847	p31_p279star	Q34508
Q100982908	p31_p279star	Q34508
Q101077837	p31_p279star	Q34508
Q101079766	p31_p279star	Q34508
Q101094418	p31_p279star	Q34508
Q101243034	p31_p279star	Q34508
Q101246930	p31_p279star	Q34508
Q101246967	p31_p279star	Q34508
grep: write error

gzip: stdout: Broken pipe


2) Check for class added by-instance, i.e. (Fireball, 'Q5451712'), (whisky, 'Q281')

In [21]:
!echo 'fireball'
!zgrep 'Q5451712' {temp}/items.remove2.tsv.gz | head

!echo 'whisky'
!zgrep 'Q281' {temp}/items.remove2.tsv.gz | head

fireball
Q5451712	p31_p279star	Q281
whisky
Q1350656	p31_p279star	Q281
Q20713240	p31_p279star	Q281
Q2535077	p31_p279star	Q281
Q536976	p31_p279star	Q281
Q7991845	p31_p279star	Q281
Q628737	p31_p279star	Q281
Q5533715	p31_p279star	Q281
Q16259546	p31_p279star	Q281
Q96278979	p31_p279star	Q281
Q982891	p31_p279star	Q281
grep: write error

gzip: stdout: Broken pipe


3) Check for class added by-outdegree, i.e. (??, 'Q100000030')

In [22]:
!zgrep 'Q100000030' {temp}/items.remove2.tsv.gz | head

Q100000030	p31_p279star	Q1357761
Q100000030	p31_p279star	Q14745
Q100000030	p31_p279star	Q14748
Q100000030	p31_p279star	Q15401930
Q100000030	p31_p279star	Q15621286
Q100000030	p31_p279star	Q16686448
Q100000030	p31_p279star	Q17537576
Q100000030	p31_p279star	Q223557
Q100000030	p31_p279star	Q2424752
Q100000030	p31_p279star	Q28877


Collect all the classes of items we will remove, just as a sanity check

In [23]:
!{kypher} -i {temp}/items.remove2.tsv.gz \
--match '()-[]->(n2)' \
--return 'count(distinct n2)' \

!{kypher} -i {temp}/items.remove2.tsv.gz \
--match '()-[]->(n2)' \
--return 'distinct n2' \
--limit 10

[2021-02-11 14:12:21 sqlstore]: DROP graph data table graph_10 from /nas/home/mbmann/subset/temp.output/items.remove2.tsv.gz
[2021-02-11 14:12:24 sqlstore]: IMPORT graph directly into table graph_10 from /nas/home/mbmann/subset/temp.output/items.remove2.tsv.gz ...
[2021-02-11 14:12:37 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_10_c1."node2")
     FROM graph_10 AS graph_10_c1
  PARAS: []
---------------------------------------------
count(DISTINCT graph_10_c1."node2")
61556
[2021-02-11 14:14:09 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node2"
     FROM graph_10 AS graph_10_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
node2
Q15075508
Q208385
Q281
Q34508
Q44
Q1357761
Q14745
Q14748
Q15401930
Q15621286


## [TODO] Create a list of all items to protect

## Create the reduced edges file

### Remove the items from the all.tsv and the label, alias and description files
We will be left with `reduced` files where the edges do not have the unwanted items. We have to remove them from the node1 and node2 positions, so we need to run the ifnotexists commands twice.

Before we start preview the files to see the column headings and check whether they look sorted.

In [24]:
!{kgtk} sort2 -i {temp}/items.remove2.tsv.gz -o {temp}/items.remove2.sorted.tsv.gz


real	0m24.116s
user	0m24.019s
sys	0m1.801s


In [25]:
!zcat < {temp}/items.remove2.sorted.tsv.gz | head | col
# !zgrep 'Q34508' {temp}/items.remove.sorted.tsv.gz -c #466

node1	label	node2
Q100000003	p31_p279star	Q34508
Q100000011	p31_p279star	Q34508
Q100000014	p31_p279star	Q34508
Q100000021	p31_p279star	Q34508
Q100000029	p31_p279star	Q34508
Q100000030	p31_p279star	Q1357761
Q100000030	p31_p279star	Q14745

gzip: Q100000030	p31_p279star	Q14748
Q100000030	p31_p279star	Q15401930
stdout: Broken pipe


Remove from the full set of edges those edges that have a `node1` present in `items.remove.sorted.tsv`

In [27]:
!zcat {temp}/items.remove2.tsv.gz | head

node1	label	node2
Q1000737	p31_p279star	Q15075508
Q1017471	p31_p279star	Q15075508
Q10350781	p31_p279star	Q15075508
Q10355535	p31_p279star	Q15075508
Q10655045	p31_p279star	Q15075508
Q10774749	p31_p279star	Q15075508
Q1092501	p31_p279star	Q15075508
Q10999547	p31_p279star	Q15075508
Q1104762	p31_p279star	Q15075508

gzip: stdout: Broken pipe


In [29]:
!{kgtk} ifnotexists -i {claims} -o {temp}/item.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted 


real	5m48.354s
user	5m46.803s
sys	0m0.782s


From the remaining edges, remove those that have a `node2` present in `items.remove.sorted.tsv`

In [30]:
!{kgtk} sort2 -i {temp}/item.edges.reduced.tsv.gz -o {temp}/item.edges.reduced.sorted.tsv.gz \
--columns node2 label node1 id


real	2m16.269s
user	2m21.052s
sys	0m9.241s


In [31]:
!{kgtk} ifnotexists -i {temp}/item.edges.reduced.sorted.tsv.gz -o {temp}/item.edges.reduced.2.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node2 \
--filter-keys node1 \
--presorted 


real	5m40.941s
user	5m39.642s
sys	0m0.499s


Create a file with the labels

In [38]:
!{kgtk} ifnotexists -i {labels} -o {temp}/label.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted


real	1m27.645s
user	1m23.328s
sys	0m0.213s


In [39]:
!{kgtk} sort2 -i {temp}/label.edges.reduced.tsv.gz -o {out}/labels.tsv.gz


real	0m11.891s
user	0m11.125s
sys	0m0.956s


Create a file with the aliases

In [40]:
!{kgtk} ifnotexists -i {aliases} -o {temp}/alias.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted


real	0m55.210s
user	0m52.657s
sys	0m0.143s


Create a file with the descriptions

In [41]:
!{kgtk} ifnotexists -i {descriptions} -o {temp}/description.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted


real	1m8.524s
user	1m5.242s
sys	0m0.205s


### Produce the output files for claims, labels, aliases and descriptions

In [42]:
!{kgtk} sort2 -i {temp}/item.edges.reduced.2.tsv.gz -o {out}/claims.tsv.gz 


real	1m49.634s
user	1m48.977s
sys	0m8.109s


In [43]:
!{kgtk} sort2 -i {temp}/label.edges.reduced.tsv.gz -o {out}/labels.en.tsv.gz 


real	0m11.444s
user	0m10.630s
sys	0m0.884s


In [44]:
!{kgtk} sort2 -i {temp}/alias.edges.reduced.tsv.gz -o {out}/aliases.en.tsv.gz 


real	0m4.116s
user	0m3.818s
sys	0m0.354s


In [45]:
!{kgtk} sort2 -i {temp}/description.edges.reduced.tsv.gz -o {out}/descriptions.en.tsv.gz 


real	0m6.953s
user	0m6.494s
sys	0m0.746s


## Tests: Confirm items were removed from claims file
**NOTE:** We will check the items we removed do not exist in claims file

1) Confirm no instance of class added manually, i.e. (class: 'Q34508')<-(instance: 'Q100431477')

In [47]:
!zgrep 'Q100431477' {out}/claims.tsv.gz #PASS: No result

2) Confirm no target_instance of class added by source_instance, i.e. (source_instance, 'Q5451712')->(class:'Q281')<-(target_instance: Q1350656)

In [49]:
!zgrep 'Q1350656	' {out}/claims.tsv.gz #PASS: No result

3) Confirm no instance with out-degree < 2, i.e. (instance: 'Q100000030')

In [50]:
!zgrep 'Q100000030	' {out}/claims.tsv.gz #PASS: No result

## Create the reduced qualifiers file
We do this by finding all the ids of the reduced edges file, and then selecting out from `qual.tsv`

We need to join by id, so we need to sort both files by id, node1, label, node2:

- `{quals}` 
- `{out}/claims.tsv.gz` 

In [51]:
!zcat < {quals} | head | column -t -s $'\t' 

id                                               node1                             label  node2                          node2;wikidatatype
P10-P1855-Q7378-555592a4-0-P10-8a982d-0          P10-P1855-Q7378-555592a4-0        P10    "Elephants Dream (2006).webm"  commonsMedia
P1000-P1896-f63a36-b84f3cd2-0-P1476-bf511b-0     P1000-P1896-f63a36-b84f3cd2-0     P1476  'FAI records'@en               monolingualtext
P1001-P1855-Q29868931-76b67d84-0-P1001-Q11736-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q11736                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q17269-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q17269                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q21208-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q21208                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q34800-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q34800                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q41079-0  

Run `ifexists` to select out the quals for the edges in `{out}/wikidataos.qual.tsv.gz`. Note that we use `node1` in the qualifier file, matching to `id` in the `wikidataos.all.tsv` file.

In [52]:
!$kgtk ifexists -i {quals} -o {out}/qualifiers.tsv.gz \
--filter-on {out}/claims.tsv.gz \
--input-keys node1 \
--filter-keys id \
--presorted


real	2m11.159s
user	2m6.879s
sys	0m0.410s


Look at the final output for qualifiers

In [53]:
!zcat < {out}/qualifiers.tsv.gz | head | col

id	node1	label	node2	node2;wikidatatype
P10-P1855-Q7378-555592a4-0-P10-8a982d-0 P10-P1855-Q7378-555592a4-0	P10	"Elephants Dream (2006).webm"	commonsMedia
P1000-P1896-f63a36-b84f3cd2-0-P1476-bf511b-0	P1000-P1896-f63a36-b84f3cd2-0	P1476	'FAI records'@en	monolingualtext
P1001-P1855-Q29868931-76b67d84-0-P1001-Q11736-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q11736	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q17269-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q17269	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q21208-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q21208	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q34800-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q34800	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q41079-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q41079	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q42392-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q42392	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q43684-0 P1001-P1855-Q29868931-76b67d84-0	P10

### Call partition and useful_files notebooks, to generate the file output files 

In [87]:
kgtk_scripts_path = "/nas/home/mbmann/kgtk_subset/kgtk"
os.environ["EXAMPLES_DIR"] = kgtk_scripts_path + "/examples"
os.environ["USECASE_DIR"] = kgtk_scripts_path + "/use-cases"
os.environ["TEMP"] = temp
os.environ["OUT"] = out
os.environ["DATATYPES"] = datatypes
os.environ["METADATA"] = metadata

In [88]:
os.environ["EXAMPLES_DIR"]

'/nas/home/mbmann/kgtk_subset/kgtk/examples'

In [79]:
!ls "$TEMP"
# !ls {temp}

alias.edges.reduced.tsv.gz	  items.remove2.sorted.tsv.gz
classes.remove2.tsv.gz		  items.remove2.tsv.gz
classes.remove.manual.tsv.gz	  items.remove.byclass.tsv.gz
classes.remove.p279.tsv.gz	  items.remove.bydegree.tsv.gz
classes.remove.p31.tsv.gz	  items.remove.tsv.gz
classes.remove.tsv.gz		  items.small.outdegree.tsv.gz
description.edges.reduced.tsv.gz  label.edges.reduced.tsv.gz
item.edges.reduced.2.tsv.gz	  metadata.out_degree.tsv.gz
item.edges.reduced.sorted.tsv.gz  subclasses.remove.tsg.gz
item.edges.reduced.tsv.gz


In [80]:
!ls "$OUT"
# !ls {out}

aliases.en.tsv.gz	labels.en.tsv.gz   wikidata.sqlite3.db
claims.tsv.gz		labels.tsv.gz
descriptions.en.tsv.gz	qualifiers.tsv.gz


**Concatenate all output files together** <br>

**NOTE:** The `metadata.property.datatypes` and `metadata.types` are not currently generated by this notebook, and have been copied from `wikidata-20200803-v5/data`. <br>
**TODO:** We must confirm if these are source files, or computed. If they are computed, we should compute them in this notebook.

In [81]:
!{kgtk} cat \
-i {out}/aliases.en.tsv.gz \
-i {out}/descriptions.en.tsv.gz \
-i {out}/qualifiers.tsv.gz \
-i {out}/claims.tsv.gz \
-i {out}/labels.en.tsv.gz \
-i {datatypes} \
-i {metadata} \
-o {out}/all.tsv.gz


real	19m58.174s
user	19m54.510s
sys	0m0.911s


In [89]:
!ls {os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb"}

/nas/home/mbmann/kgtk_subset/kgtk/examples/partition-wikidata.ipynb


In [90]:
pm.execute_notebook(
    os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb",
    os.environ["TEMP"] + "/partition-wikidata.out.ipynb",
    parameters=dict(
        wikidata_input_path = os.environ["OUT"] + "/all.tsv.gz",
        wikidata_parts_path = os.environ["OUT"] + "/parts",
        temp_folder_path = os.environ["OUT"] + "/parts/temp",
        sort_extras = "--buffer-size 30% --temporary-directory $OUT/parts/temp",
        verbose = False
    )
)
;

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
pm.execute_notebook(
    os.environ["USECASE_DIR"] + "/Wikidata Useful Files.ipynb",
    os.environ["TEMP"] + "/Wikidata Useful Files Out.ipynb",
    parameters=dict(
        output_path = os.environ["OUT"],
        output_folder = "useful_files",
        temp_folder = "temp.useful_files",
        wiki_root_folder = os.environ["OUT"] + "/parts/",
        cache_path = os.environ["OUT"] + "/temp.useful_files",
        languages = 'en',
        compute_pagerank = True,
        delete_database = False
    )
)
;

## Sanity checks

- After removing classes, check that the class does not occur in the resulting claims file. 
- After protecting classes, check the class was occurs in the resulting claims file.

Check that the class we removed `Q34508` was removed from claims file

In [ ]:
!{kypher} -i {out}/claims.tsv.gz \
--match '(n1:Q34508)-[l]->(n2)' \
--limit 10 \
| col

In [ ]:
!{kypher} -i {out}/claims.tsv.gz \
--match '(n1:P131)-[l]->(n2)' \
--limit 10 \
| col

## Compute the derived files using the `Wikidata Useful Files` Jupyter notebook

Compute `claims.wikibase-item.tsv.gz` which would be computed by the Wikidata partitioner, but we are not using it here yet

In [ ]:
!zcat < "{datatypes}" | head | col

In [ ]:
!{kypher} -i {out}/claims.tsv.gz -i "{datatypes}" -o {out}/claims.wikibase-item.tsv.gz \
--match 'claims: (n1)-[l {label: p}]->(n2), datatypes: (p)-[:datatype]->(:`wikibase-item`)' \
--return 'l as id, n1 as node1, p as label, n2 as node2' \
--order-by 'l' 

To compute the derived files we use papermill to run the `Wikidata Useful Files` notebook.

In [ ]:
pm.execute_notebook(
    notebooks_folder + useful_files_notebook,
    temp + "/useful_files_notebook_output.ipynb",
    parameters=dict(
        output_path=output_path,
        output_folder=output_folder,
        temp_folder=temp_folder,
        wiki_root_folder=wiki_root_folder,
        claims_file="claims.tsv.gz",
        label_file="labels.en.tsv.gz",
        alias_file="aliases.en.tsv.gz",
        description_file="descriptions.en.tsv.gz",
        item_file="claims.wikibase-item.tsv.gz",
        cache_path=cache_path,
        delete_database=delete_database,
        compute_pagerank=compute_pagerank
    )
)

Look at the columns so we know how to construct the kypher query

## Summary of results

In [ ]:
!ls -lh {out}/*wikidataos.*

In [ ]:
!zcat < {out}/wikidataos.all.tsv.gz | wc

## Verification

The edges file must contain edges for properties, this is not the case on 2020-11-10`


In [ ]:
!{kgtk} -i "{claims}" \
--match '(:P10)-[l]->(n2)' \
--limit 10

## concatenate files to get the `all` file

In [ ]:
lad = []
if 'en' not in languages:
    languages.append('en')
for lang in languages:
    lad.append(f"{out}/labels.{lang}.tsv.gz")
    lad.append(f"{out}/aliases.{lang}.tsv.gz")
    lad.append(f"{out}/descriptions.{lang}.tsv.gz")
lad_file_list = " ".join(lad)

In [ ]:
!kgtk cat -i {out}/claims.tsv.gz \
{lad_file_list} \
{out}/qualifiers.tsv.gz \
{out}/metadata.pagerank.undirected.tsv.gz \
{out}/metadata.pagerank.directed.tsv.gz \
{out}/metadata.in_degree.tsv.gz \
{out}/metadata.out_degree.tsv.gz \
-o {out}/wikidataos.all.tsv.gz

## concatenate files to get the `all for triples` file


In [ ]:
!kgtk cat -i $OUT/wikidataos.all.tsv.gz \
$OUT/derived.P31.tsv.gz \
$OUT/derived.P279.tsv.gz \
$OUT/derived.isa.tsv.gz \
$OUT/derived.P279star.tsv.gz \
-o $OUT/wikidataos.all.for.triples.tsv.gz

## Filter out `novalue`, `somevalue` and `P9`

In [ ]:
!kgtk filter -i $OUT/wikidataos.all.for.triples.tsv.gz \
    -o $OUT/wikidataos.all.for.triples.filtered.tsv.gz \
    -p ';;somevalue,novalue,P9' --invert

## Add ids for any edge with missing id

In [ ]:
!kgtk add-id -i $OUT/wikidataos.all.for.triples.filtered.tsv.gz \
-o $OUT/wikidataos.all.for.triples.filtered.id.tsv.gz \
--id-style wikidata

## Sort by `id`

In [ ]:
!kgtk sort2 -i $OUT/wikidataos.all.for.triples.filtered.id.tsv.gz \
-o $OUT/wikidataos.all.for.triples.filtered.id.sorted.tsv.gz 
-c id